In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
from sklearn import model_selection, cross_validation
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import *
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

In [5]:
train_csv = pd.read_csv('train_cap2018.csv')

In [43]:
# get the number of missing data points per column
missing_values_count = train_csv.isnull().sum()

#percent of the data missing for columns containing missing values
missing_values_count[missing_values_count != 0] / train_csv.shape[0] *100

MATTR    80.010985
MSTTR    79.223728
dtype: float64

In [44]:
train_cols = train_csv.columns.tolist()
#we drop high missing values columns
train_cols.remove("MATTR")
train_cols.remove("MSTTR")
train_cols.remove("text")
train_cols.remove("fulltext")
train_cols.remove("level1")
target = "level1"

Y = train_csv[target].astype('str')
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_csv[train_cols], label_encoded_y, test_size=test_size, random_state=seed)
# fit model no training data

parameters = {'objective':'multi:softprob',
              'n_jobs':6,
              'n_thread': 4,
              'num_class': 6,
              'learning_rate': 0.01, #so called `eta` value
              'max_depth': 8,
              'min_child_weight': 11,
              'silent': [1],
              'subsample': 0.7,
              'colsample_bytree': 0.7,
              'n_estimators': 1000, #number of trees, change it to 1000 for better results
              'seed': 1337}

xgtrain = xgboost.DMatrix(train_csv[train_cols], label=label_encoded_y)
model = xgboost.XGBClassifier(**parameters)
xgb_param = model.get_xgb_params()
#do cross validation
print ('Start cross validation')
cvresult = xgboost.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=15, metrics=['merror'],
     early_stopping_rounds=50, stratified=True, seed=1301)

print('Best number of trees = {}'.format(cvresult.shape[0]))
model.set_params(n_estimators=cvresult.shape[0])
print('Fit on the trainingsdata')
model.fit(train_csv[train_cols], label_encoded_y, eval_metric='merror')

Start cross validation


In [ ]:
print(model)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))